In [1]:
!pip install onnx


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import onnx
from onnx import helper

model = onnx.load("experiments\hand_landmark_full.onnx")


In [20]:
target_layer_name = 'model_1/model/batch_normalization/FusedBatchNormV3'
target_layer_shape = None

for node in model.graph.node:
    # Split the output names by semicolon
    output_names = node.output[0].split(';')
    if target_layer_name in output_names:
        # Search for the target layer name in value_info
        for output_tensor in model.graph.value_info:
            if output_tensor.name == target_layer_name:
                target_layer_shape = [d.dim_value for d in output_tensor.type.tensor_type.shape.dim]
                break
        # If not found in value_info, check initializer
        if target_layer_shape is None:
            for init in model.graph.initializer:
                if init.name == target_layer_name:
                    target_layer_shape = init.dims
                    break
        if target_layer_shape is not None:
            break


In [19]:
model.graph.node[0]

input: "input_1"
input: "model_1/model/conv2d/Conv2D"
input: "model_1/model/batch_normalization/FusedBatchNormV3"
output: "TFLITE2ONNX_FAF_model_1/model/re_lu/Relu6;model_1/model/batch_normalization/FusedBatchNormV3;model_1/model/batch_normalization_1/FusedBatchNormV3;model_1/model/depthwise_conv2d/depthwise;model_1/model/conv2d_5/Conv2D;model_1/model/conv2d/Conv2D"
name: "TFLITE2ONNX_FAF_model_1/model/re_lu/Relu6;model_1/model/batch_normalization/FusedBatchNormV3;model_1/model/batch_normalization_1/FusedBatchNormV3;model_1/model/depthwise_conv2d/depthwise;model_1/model/conv2d_5/Conv2D;model_1/model/conv2d/Conv2D"
op_type: "Conv"
attribute {
  name: "dilations"
  type: INTS
  ints: 1
  ints: 1
}
attribute {
  name: "group"
  type: INT
  i: 1
}
attribute {
  name: "kernel_shape"
  type: INTS
  ints: 3
  ints: 3
}
attribute {
  name: "pads"
  type: INTS
  ints: 0
  ints: 0
  ints: 1
  ints: 1
}
attribute {
  name: "strides"
  type: INTS
  ints: 2
  ints: 2
}

In [21]:
if target_layer_shape is None:
    print(f"Could not find the shape of '{target_layer_name}' in the ONNX graph.")
else:
    print(f"Shape of '{target_layer_name}': {target_layer_shape}")

    # Add the target layer as an output to the model
    intermediate_layer = helper.make_tensor_value_info(
        name=target_layer_name,
        elem_type=onnx.TensorProto.FLOAT,
        shape=target_layer_shape
    )
    model.graph.output.append(intermediate_layer)

    # Save the modified model
    onnx.save(model, "model_modified.onnx")
    print(f"Added '{target_layer_name}' as an output. Modified model saved to 'model_modified.onnx'")

Shape of 'model_1/model/batch_normalization/FusedBatchNormV3': [24]
Added 'model_1/model/batch_normalization/FusedBatchNormV3' as an output. Modified model saved to 'model_modified.onnx'


In [22]:
import onnxruntime as ort
import numpy as np
from PIL import Image
import cv2

def preprocess_image(image_path, input_shape):
    inp = Image.open(image_path)
    inp = inp.convert("RGB")
    resized_image = inp.resize((input_shape[2], input_shape[3]))
    input_data = np.array(resized_image, dtype=np.float32)
    input_data = np.transpose(input_data, (2, 0, 1))  # Change data layout from HWC to CHW
    input_data = np.expand_dims(input_data, axis=0)  # Add batch dimension
    input_data = input_data / 255.0
    return input_data
def run_inference(model_path, image_path):
    """
    Runs inference on a single image using the ONNX model.

    Args:
        model_path: Path to the ONNX model file.
        image_path: Path to the input image.

    Returns:
        A dictionary where keys are output names and values are the corresponding outputs.
    """
    # Create an ONNX Runtime session
    ort_session = ort.InferenceSession(model_path)

    # Get model input and output details
    model_inputs = ort_session.get_inputs()
    input_names = [input.name for input in model_inputs]
    input_shape = model_inputs[0].shape  # Assuming single input

    model_outputs = ort_session.get_outputs()
    output_names = [output.name for output in model_outputs]

    # Preprocess the image
    input_data = preprocess_image(image_path, input_shape)

    # Run inference
    outputs = ort_session.run(output_names, {input_names[0]: input_data})

    # Create a dictionary of outputs
    output_dict = {name: output for name, output in zip(output_names, outputs)}

    return output_dict

In [23]:
image_path = "experiments/palm_rect.jpg"
out = run_inference("model_modified.onnx", image_path)


In [28]:
model_path = "model_modified.onnx"
model = onnx.load(model_path)
graph = model.graph

# Iterate through the BatchNormalization nodes in the graph
typess = []
for node in graph.node:
    typess.append(node.op_type)
    if node.op_type == 'BatchNormalization':
        print(node.name)


In [5]:
graph.node[0]

NameError: name 'graph' is not defined

In [18]:
import onnx

def save_graph_to_txt(graph, file_path):
    """
    Saves an ONNX graph to a text file, describing its structure without weights.

    Args:
        graph: The ONNX graph (onnx.onnx_ml_pb2.GraphProto).
        file_path: The path to the output text file.
    """

    with open(file_path, "w") as f:
        f.write("Graph Name: {}\n".format(graph.name))

        f.write("\nInputs:\n")
        for input_ in graph.input:
            f.write(get_value_info_str(input_))

        f.write("\nOutputs:\n")
        for output in graph.output:
            f.write(get_value_info_str(output))

        f.write("\nNodes:\n")
        for node in graph.node:
            f.write(get_node_str(node))

def get_value_info_str(value_info):
    """
    Generates a string representation of an ONNX ValueInfoProto.

    Args:
        value_info: The ONNX ValueInfoProto.

    Returns:
        A string describing the ValueInfoProto.
    """
    
    string = "  Name: {}\n".format(value_info.name)
    
    tensor_type = value_info.type.tensor_type
    string += "    Type: {}\n".format(onnx.TensorProto.DataType.Name(tensor_type.elem_type))

    shape = tensor_type.shape
    string += "    Shape: {}\n".format([dim.dim_value for dim in shape.dim])
    
    return string

def get_node_str(node):
    """
    Generates a string representation of an ONNX NodeProto.

    Args:
        node: The ONNX NodeProto.

    Returns:
        A string describing the NodeProto.
    """

    string = "  Op Type: {}\n".format(node.op_type)
    string += "  Name: {}\n".format(node.name)

    string += "    Inputs:\n"
    for input_name in node.input:
        string += "      {}\n".format(input_name)

    string += "    Outputs:\n"
    for output_name in node.output:
        string += "      {}\n".format(output_name)

    string += "    Attributes:\n"
    for attr in node.attribute:
        string += "      {}: {}\n".format(attr.name, get_attribute_value(attr))

    return string

def get_attribute_value(attr):
    """
    Extracts the value from an ONNX AttributeProto, excluding weights.

    Args:
        attr: The ONNX AttributeProto.

    Returns:
        The attribute value (without weights).
    """

    if attr.type == onnx.AttributeProto.FLOAT:
        return attr.f
    elif attr.type == onnx.AttributeProto.INT:
        return attr.i
    elif attr.type == onnx.AttributeProto.STRING:
        return attr.s.decode('utf-8')
    elif attr.type == onnx.AttributeProto.TENSOR:
        # Exclude weights (tensors)
        return "Tensor (excluded)"
    elif attr.type == onnx.AttributeProto.GRAPH:
        return "Sub-graph (excluded)"
    elif attr.type == onnx.AttributeProto.FLOATS:
        return list(attr.floats)
    elif attr.type == onnx.AttributeProto.INTS:
        return list(attr.ints)
    elif attr.type == onnx.AttributeProto.STRINGS:
        return [s.decode('utf-8') for s in attr.strings]
    elif attr.type == onnx.AttributeProto.TENSORS:
        return "Tensors (excluded)"
    elif attr.type == onnx.AttributeProto.GRAPHS:
        return "Sub-graphs (excluded)"
    else:
        return "Unknown attribute type"

# Example usage (assuming you have an ONNX model loaded):
model_path = "experiments\hand_landmark_full.onnx"  # Replace with your model path
model = onnx.load(model_path)
graph = model.graph

#save_graph_to_txt(graph, "graph_description.txt")
#print("Successfully saved the graph description to graph_description.txt")

In [15]:
graph.node[0]


['input_1', 'model_1/model/conv2d/Conv2D', 'model_1/model/batch_normalization/FusedBatchNormV3']

In [47]:
with open("node_info.txt", "w") as f:
    f.write(graph.node[0].__str__())

In [17]:
from tf_hand_landmark.hl_source import create_hand_landmark_model

model = create_hand_landmark_model()
model.summary()



Model: "hand_landmark_model"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_1             │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ model_1_model_conv… │ (None, 112, 112,  │        432 │ input_1[0][0]     │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ model_1_model_batc… │ (None, 112, 112,  │         64 │ model_1_model_co… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ model_1_model_re_l… │ (None, 112, 112,  │          0 │ model_1_model_ba… │
│ (ReLU)              │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ model_1_model_dept… │ (None, 112, 112,  │        144 │ model_1_model_re… │
│ (DepthwiseConv2D)   │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ model_1_model_batc… │ (None, 112, 112,  │         64 │ model_1_model_de… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ model_1_model_re_l… │ (None, 112, 112,  │          0 │ model_1_model_ba… │
│ (ReLU)              │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ model_1_model_conv… │ (None, 112, 112,  │        256 │ model_1_model_re… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ model_1_model_batc… │ (None, 112, 112,  │         64 │ model_1_model_co… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ model_1_model_dept… │ (None, 56, 56,    │        144 │ model_1_model_ba… │
│ (DepthwiseConv2D)   │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ model_1_model_batc… │ (None, 56, 56,    │         64 │ model_1_model_de… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ model_1_model_re_l… │ (None, 56, 56,    │          0 │ model_1_model_ba… │
│ (ReLU)              │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ model_1_model_conv… │ (None, 56, 56,    │        512 │ model_1_model_re… │
│ (Conv2D)            │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ model_1_model_batc… │ (None, 56, 56,    │        128 │ model_1_model_co… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ model_1_model_dept… │ (None, 56, 56,    │        288 │ model_1_model_ba… │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ model_1_model_batc… │ (None, 56, 56,    │        128 │ model_1_model_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ model_1_model_re_l… │ (None, 56, 56,    │          0 │ model_1_model_ba

 Total params: 4,936,017 (18.83 MB)

 Trainable params: 4,909,009 (18.73 MB)

 Non-trainable params: 27,008 (105.50 KB)

In [21]:
import numpy as np
onnx_weights = {}
for initializer in model.graph.initializer:
    onnx_weights[initializer.name] = onnx.numpy_helper.to_array(initializer)
    

In [30]:
list(onnx_weights.items())[0]


('model_1/model/batch_normalization_33/FusedBatchNormV3',
 array([-3.98193359e-01, -1.01745605e-01,  7.52639771e-03,  4.01367188e-01,
        -2.31201172e-01, -1.03027344e-01,  7.20214844e-03, -2.26593018e-02,
        -2.30834961e-01, -3.33740234e-01, -2.81982422e-01, -5.24902344e-01,
        -1.52832031e-01, -3.64013672e-01, -3.23730469e-01, -7.27539062e-01,
        -3.16406250e-01, -6.65893555e-02,  8.75244141e-02,  3.70361328e-01,
        -2.25585938e-01,  4.80712891e-01,  3.22509766e-01,  1.20788574e-01,
        -5.96679688e-01, -2.21923828e-01, -2.15606689e-02, -3.43261719e-01,
        -9.85717773e-02, -3.44238281e-01, -7.22656250e-01, -4.42382812e-01,
        -2.14721680e-01, -1.55273438e-01, -3.83056641e-01,  4.66613770e-02,
        -7.16796875e-01, -1.56738281e-01, -2.85156250e-01, -3.94287109e-01,
         5.04302979e-03, -1.65771484e-01, -2.23022461e-01,  2.28027344e-01,
        -7.21679688e-01, -3.13476562e-01, -2.21923828e-01, -2.94433594e-01,
        -2.01171875e-01, -2.27

In [31]:
with open("weights_info.txt", "w") as f:
    for key, weight in onnx_weights.items():
        f.write(f"Key: {key}, Shape: {weight.shape}, Type: {weight.dtype}\n")
